<a href="https://colab.research.google.com/github/WhiteJaeger/googleColab/blob/master/Lab_2%2C_IAD%2C_Kashchikhin_Andrej.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [75]:
!ls -la ./drive/My\ Drive/Colab\ Notebooks/All-seasons.csv

-rw------- 1 root root 5533363 Sep 20  2019 './drive/My Drive/Colab Notebooks/All-seasons.csv'


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
data = pd.read_csv('./drive/My Drive/Colab Notebooks/All-seasons.csv')

In [98]:
data.head()

,Season,Episode,Character,Line
0,10,1,Stan,"You guys, you guys! Chef is going away. \n"
1,10,1,Kyle,Going away? For how long?\n
2,10,1,Stan,Forever.\n
3,10,1,Chef,I'm sorry boys.\n
4,10,1,Stan,"Chef said he's been bored, so he joining a gro..."


In [99]:
data.shape

(70896, 4)

In [0]:
data = data.drop(columns=['Season', 'Episode'])

In [101]:
data.Character.value_counts()

Cartman            9774
Stan               7680
Kyle               7099
Butters            2602
Randy              2467
                   ... 
Evan Stone            1
Captain Bly           1
Nick Nolte            1
Centers               1
Ethiopian Woman       1
Name: Character, Length: 3950, dtype: int64

In [102]:
print(data.Character.isna().sum())

print(data.Character.isnull().sum())

0
0


In [103]:
print(data.Line.isna().sum())

print(data.Line.isnull().sum())

0
0


In [104]:
data.describe()

,Character,Line
count,70896,70896
unique,3950,64301
top,Cartman,What?\n
freq,9774,361


In [105]:
data['numLines'] = data.groupby('Character')['Line'].transform('size')

data.head()

,Character,Line,numLines
0,Stan,"You guys, you guys! Chef is going away. \n",7680
1,Kyle,Going away? For how long?\n,7099
2,Stan,Forever.\n,7680
3,Chef,I'm sorry boys.\n,917
4,Stan,"Chef said he's been bored, so he joining a gro...",7680


In [106]:
data.shape

(70896, 3)

In [0]:
data = data[data['numLines'] > 800]

In [108]:
data.shape

(33284, 3)

In [0]:
import re

def cleanText(text: str) -> str:

    text = text.lower()
    
    text = re.sub(r"\n", "",  text)
    text = re.sub(r"[-()]", "", text)
    text = re.sub(r"\.", " .", text)
    text = re.sub(r"\!", " !", text)
    text = re.sub(r"\?", " ?", text)
    text = re.sub(r"\,", " ,", text)
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"ohh", "oh", text)
    text = re.sub(r"ohhh", "oh", text)
    text = re.sub(r"ohhhh", "oh", text)
    text = re.sub(r"ohhhhh", "oh", text)
    text = re.sub(r"ohhhhhh", "oh", text)
    text = re.sub(r"ahh", "ah", text)
    text = re.sub(r"[^a-zA-Z0-9]+", ' ', text)
    
    return text

In [0]:
data['Line'] = data['Line'].apply(lambda line: cleanText(line))

In [111]:
data.head()

,Character,Line,numLines
0,Stan,you guys you guys chef is going away,7680
1,Kyle,going away for how long,7099
2,Stan,forever,7680
3,Chef,i am sorry boys,917
4,Stan,chef said he is been bored so he joining a gro...,7680


In [125]:
!pip install nltk

import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
tokenizer = nltk.tokenize.WordPunctTokenizer()

stopWords = nltk.corpus.stopwords.words()

In [0]:
from nltk.stem.lancaster import LancasterStemmer

stemmer = LancasterStemmer()

In [135]:
stemmer.stem('guys')

'guy'

In [0]:
data['Line'] = data['Line'].apply(lambda line: tokenizer.tokenize(line))
data['Line'] = data['Line'].apply(lambda line: ' '.join(line))

In [0]:
def removeStopWords(sentence: str) -> str:
  cleanedSentence = [word for word in sentence.split() if word not in stopWords]
  return ' '.join(cleanedSentence)

In [0]:
data['Line'] = data['Line'].apply(removeStopWords)

In [140]:
data.head()

,Character,Line,numLines
0,8,guy guy chef going away,7680
1,4,going away long,7099
2,8,forev,7680
3,2,sorry boy,917
4,8,chef said bor join group cal sup adv club,7680


In [133]:
data.Character.value_counts()

1    9774
8    7680
4    7099
0    2602
6    2467
5    1002
2     917
3     881
7     862
Name: Character, dtype: int64

In [0]:
from sklearn.preprocessing import LabelEncoder

In [118]:
le = LabelEncoder()
character = le.fit_transform(data['Character'])
print(np.unique(character))

[0 1 2 3 4 5 6 7 8]


In [0]:
data['Character'] = character

In [120]:
data.head()

,Character,Line,numLines
0,8,you guys you guys chef is going away,7680
1,4,going away for how long,7099
2,8,forever,7680
3,2,i am sorry boys,917
4,8,chef said he is been bored so he joining a gro...,7680


In [0]:
allLines = [line for line in data.Line]

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(oov_token='<OOV>')

In [0]:
tokenizer.fit_on_texts(allLines)

In [0]:
lines = [tokenizer.texts_to_sequences([sentence])[0] for sentence in data['Line']]

In [151]:
print('\nLine: ', data['Line'][0],'\nLine in sequence format: ', lines[0])

print()

print('\nLine: ', data['Line'][20],'\nLine in sequence format: ', lines[20])


Line:  guy guy chef going away 
Line in sequence format:  [6, 6, 120, 30, 100]


Line:  revers jew 
Line in sequence format:  [35, 39]


In [0]:
dataBowRaw = data.copy()

dataBowRaw['Line'] = lines

In [160]:
dataBowRaw.head()

,Character,Line,numLines
0,8,"[6, 6, 120, 30, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0...",7680
1,4,"[30, 100, 128, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",7099
2,8,"[564, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",7680
3,2,"[86, 63, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",917
4,8,"[120, 70, 687, 477, 812, 80, 455, 498, 538, 0,...",7680


In [161]:
maxLength = max([len(sentence) for sentence in dataBowRaw['Line']])
maxLength

143

In [0]:
lineBow = []

for line_ in dataBow['Line']:
  if len(line_) < maxLength: # Нужно, чтобы все данные были в одном формате - одинаковой длины.
    for i in range(maxLength - len(line_)):
      line_.append(0) # Если меньше - добавим нули.
  lineBow.append(line_)

In [163]:
dataBowRaw['Line'] = lineBow

dataBowRaw.head()

,Character,Line,numLines
0,8,"[6, 6, 120, 30, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0...",7680
1,4,"[30, 100, 128, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",7099
2,8,"[564, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",7680
3,2,"[86, 63, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",917
4,8,"[120, 70, 687, 477, 812, 80, 455, 498, 538, 0,...",7680


In [164]:
dataBow = pd.DataFrame(np.stack(list(dataBowRaw['Line'])))
dataBow['Character'] = dataBowRaw['Character']

print(dataBow.shape)
print()
dataBow.head()

(33284, 144)



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,Character
0,6,6,120,30,100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8.0
1,30,100,128,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.0
2,564,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8.0
3,86,63,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.0
4,120,70,687,477,812,80,455,498,538,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8.0


In [0]:
dataBow.dropna(inplace=True)

In [0]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing

In [0]:
target = dataBow['Character'].values

In [0]:
def metrics_report(true_y, predicted_y) -> dict:
  mse = metrics.mean_squared_error(y_pred=predicted_y, 
                                 y_true=true_y)

  rmse = np.sqrt(mse)

  mae = metrics.mean_absolute_error(y_pred=predicted_y, 
                                  y_true=true_y)

  medae = metrics.median_absolute_error(y_pred=predicted_y, 
                                      y_true=true_y)

  R2 = metrics.r2_score(y_pred=predicted_y, 
                      y_true=true_y)
  
  return {
      'mse': mse,
      'rmse': rmse,
      'mae': mae,
      'medae': medae,
      'r2': R2
  }

In [0]:
(trainData, testData, 
trainDependentVariables, 
testDependentVariables) = train_test_split(dataBow,
                                            target,
                                            test_size=0.3, 
                                            random_state=5)

In [176]:
regressionModel = linear_model.LinearRegression()

regressionModel.fit(trainData, trainDependentVariables)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
predictedDependentVariables = regressionModel.predict(testData)

In [179]:
LRBoW = metrics_report(testDependentVariables, predictedDependentVariables)

print("MSE:   {}\nRMSE:  {}\nR2:    {}\nMAE:   {}\nMedAE: {}".format(LRBoW['mse'], LRBoW['rmse'], LRBoW['r2'], LRBoW['mae'], LRBoW['medae']))

MSE:   4.724852529795197e-20
RMSE:  2.1736725902939469e-10
R2:    1.0
MAE:   5.629167288148451e-12
MedAE: 3.552713678800501e-15
